In [ ]:
import os
import json

In [ ]:
# 方法3: split_hypodd関数を実行してindexを取得
def get_index_from_split_hypodd(
    config_json: str = "config/config.json",
    catalog_csv: str = "gamma/gamma_catalog.csv"
) -> list:
    """split_hypodd関数を実行してindex範囲を取得"""
    import pandas as pd
    
    try:
        with open(config_json, "r") as fp:
            config = json.load(fp)

        events = pd.read_csv(catalog_csv)

        if "MAXEVENT" in config["hypodd"]:
            MAXEVENT = config["hypodd"]["MAXEVENT"]
        else:
            MAXEVENT = 1e4

        MAXEVENT = len(events) // ((len(events) - 1) // MAXEVENT + 1) + 1
        num_parallel = int((len(events) - 1) // MAXEVENT + 1)
        
        index_list = list(range(num_parallel))
        print(f"Calculated index from catalog: {index_list} (total events: {len(events)}, MAXEVENT: {MAXEVENT})")
        return index_list
        
    except Exception as e:
        print(f"Error calculating index: {e}")
        print("Using default index [0]")
        return [0]

# カタログから動的にindexを計算
index = get_index_from_split_hypodd()
print(f"Calculated index: {index}")

In [ ]:
def merge_hypodd(
    index: list,
    config_json: "json",
    catalog_ct: str,
    catalog_cc: str,
    # bucket_name: str = "catalogs",
    # s3_url: str = "minio-service:9000",
    # secure: bool = False,
):
    import json
    import os
    from glob import glob

    # from minio import Minio

    # minioClient = Minio(s3_url, access_key="minio", secret_key="minio123", secure=secure)

    with open(config_json, "r") as fp:
        config = json.load(fp)

    # objects = minioClient.list_objects(bucket_name, prefix=f"{config['region']}/hypodd/hypodd_", recursive=True)

    tmp_path = lambda x: os.path.join("/tmp/", x)
    # for obj in objects:
    #     print(obj._object_name)
    #     minioClient.fget_object(bucket_name, obj._object_name, tmp_path(obj._object_name.split("/")[-1]))

    # tmp_ct_catalogs = sorted(glob(tmp_path("hypodd_ct_*.reloc")))
    hypodd_ct_catalogs = [tmp_path(f"hypodd_ct_{i:03d}.reloc") for i in index]
    print(f"cat {' '.join(hypodd_ct_catalogs)} > {tmp_path('hypodd_ct_catalog.txt')}")
    os.system(f"cat {' '.join(hypodd_ct_catalogs)} > {tmp_path('hypodd_ct_catalog.txt')}")
    # minioClient.fput_object(
    #     bucket_name,
    #     f"{config['region']}/hypodd_ct_catalog.txt",
    #     tmp_path("hypodd_ct_catalog.txt"),
    # )
    os.system(f"cat {tmp_path('hypodd_ct_catalog.txt')} > {catalog_ct}")

    # hypodd_cc_catalogs = sorted(glob(tmp_path("hypodd_cc_*.reloc")))
    hypodd_cc_catalogs = [tmp_path(f"hypodd_cc_{i:03d}.reloc") for i in index]
    print(f"cat {' '.join(hypodd_cc_catalogs)} > {tmp_path('hypodd_cc_catalog.txt')}")
    os.system(f"cat {' '.join(hypodd_cc_catalogs)} > {tmp_path('hypodd_cc_catalog.txt')}")
    # minioClient.fput_object(
    #     bucket_name,
    #     f"{config['region']}/hypodd_cc_catalog.txt",
    #     tmp_path("hypodd_cc_catalog.txt"),
    # )
    os.system(f"cat {tmp_path('hypodd_cc_catalog.txt')} > {catalog_cc}")

    return 0

In [ ]:
merge_hypodd(
    index=get_index_from_split_hypodd(),
    config_json="config/config.json",
    catalog_ct="hypodd_ct_catalog.txt",
    catalog_cc="hypodd_cc_catalog.txt",
)

In [8]:
# Kubeflow Pipelines UI用のメタデータ出力
if os.environ.get('ELYRA_RUNTIME_ENV') == 'kfp':
    # For information about Elyra environment variables refer to
    # https://elyra.readthedocs.io/en/stable/user_guide/best-practices-file-based-nodes.html#proprietary-environment-variables

    metadata = {
        'outputs': [
            {
                'storage': 'inline',
                'source': f'# Merge hypodd results Complete\n...',
                'type': 'markdown',
            }
        ]
    }

    with open('mlpipeline-ui-metadata.json', 'w') as f:
        json.dump(metadata, f)